In [3]:
import pandas as pd
import gzip
import shutil
from datetime import datetime, timedelta

# Step 1: Generate sample data (timestamp, user_id, value)
data = {
    "timestamp": pd.date_range("2025-01-01", periods=1000, freq="h"),
    "user_id": [f"user_{i%10}" for i in range(1000)],
    "value": [i % 50 for i in range(1000)]
}
df = pd.DataFrame(data)

# Introduce duplicates
df = pd.concat([df, df.iloc[:100]], ignore_index=True)

# Step 2: Apply retention policy (keep last 60 days only)
retention_period = 60
cutoff = datetime.now() - timedelta(days=retention_period)
df = df[df["timestamp"] >= cutoff]

# Step 3: Deduplicate
df = df.drop_duplicates()

# Step 4: Summarize data (daily averages per user)
df["date"] = df["timestamp"].dt.date
summary = df.groupby(["date", "user_id"])["value"].mean().reset_index()

# Step 5: Save cleaned data
summary.to_csv("cleaned_data.csv", index=False)

# Step 6: Compress with gzip
with open("cleaned_data.csv", "rb") as f_in:
    with gzip.open("cleaned_data.csv.gz", "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

print("Data cleaning and compression completed.")


Data cleaning and compression completed.
